Pull in firstpage of reports and find out how many pages there are in total

In [ ]:
import requests
from bs4 import BeautifulSoup
URL = 'https://www.gov.uk/service-standard-reports'
assessments_pg1 = requests.get(URL)
assessments_pg1_content = BeautifulSoup(assessments_pg1.content, 'html.parser')

import re
pagination_label = assessments_pg1_content.find('span',class_ = 'gem-c-pagination__link-label').text.strip()
# do I need to add something if there is just one page? probably never happen?
final_page = re.search('.* of (.*)', pagination_label, re.IGNORECASE).group(1)


Get all the individual links for all pages plus some summary information that's provided for some assessments

In [ ]:
URL = 'https://www.gov.uk/service-standard-reports?page='
i = 1
service_details = []
while i <= int(final_page):
  page = requests.get(URL+str(i))
  content = BeautifulSoup(page.content, 'html.parser')
  results = content.find_all('li', class_='gem-c-document-list__item')
  for result in results: 
          attribute_array = {"service":[],'attributes':{"url":[],"assessment_result":[],"assessment_stage":[''],"assessment_date":[''],'service_provider':[''],'note':['']}}
          title_html = result.find('a', class_='gem-c-document-list__item-title gem-c-document-list__item-link')
          title_text = title_html.text.strip()
          attribute_array['service'] = title_text
          url = result.find('a')['href']
          attribute_array['attributes']['url'] = url
          attributes = result.find_all('li', class_='gem-c-document-list__attribute')
          if attribute_array['attributes']['url'] != "/service-standard-reports/nhs-111":
            service_details.append(attribute_array)
  i += 1

In [ ]:
def get_service_attributes(content,service_detail): 
    table_results = content.find_all('tr')
    for row in table_results:
        row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
        if re.search('^Assessment date:.*', row_text, re.IGNORECASE):
            assessment_date = re.search('^(Assessment date:)(.*)', row_text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_date'] = assessment_date
        if re.search('^Stage:.*', row_text, re.IGNORECASE):
            assessment_stage = re.search('^(Stage:)(.*)', row_text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_stage'] = assessment_stage
        if re.search('^Result:.*', row_text, re.IGNORECASE):
            assessment_result = re.search('^(Result:)(.*)', row_text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_result'] = assessment_result
        if re.search('^Service provider:.*', row_text, re.IGNORECASE):
            service_provider = ''
            service_provider = re.search('^(Service provider:)(.*)', row_text, re.IGNORECASE).group(2)
            service_detail['attributes']['service_provider'] = service_provider
        else:
            service_detail['attributes']['service_provider'] = ''
        note = content.find('p',class_='gem-c-lead-paragraph').text.strip()
        service_detail['attributes']['note'] = note
    return(service_detail)

In [ ]:
from datetime import datetime
first_service_results = []
for service_detail in service_details:
    point=''
    result = ''
    collect_next_cell = ''
    service_result = {"service_details":[],'results':[],'html_contet':[]}
    extension = service_detail['attributes']['url']
    URL = 'https://www.gov.uk' + extension
    page = requests.get(URL)
    content = BeautifulSoup(page.content, 'html.parser')
    service_result['html_content'] = content 
    results = content.find_all('p')
    service_attributes = content.find_all('dd')
    for service_attribute in service_attributes:
        get_service_attributes(content,service_detail)    
# Get results from newest standard format pages
# these are reports with no table and the decisions within the text     
    service_result['service_details'] = service_detail
    for p in results:
        p_text = p.text.strip()
        if re.search('^The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE):
            result = re.search('The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE).group(2)
            point = re.search('The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE).group(3)        
            service_result['results'].append({'point':point,'result':result})
        if re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE):
            result = re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE).group(1)
            point = re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE).group(2)        
            service_result['results'].append({'point':point,'result':result})
    first_service_results.append(service_result)

In [ ]:
#Get results from older tables
# # where the links in the table are helpfully numbered
for service in first_service_results:
    if service["results"] == []:
        service_result = {"service_details":[],'results':[],'html_contet':[]}
        for service_detail in service_details:
            if service_detail['service'] == service['service_details']['service']:  
                content = service['html_content']
                get_service_attributes(content,service_detail)
                service_result['service_details'] = service_detail
                table_results = content.find_all('tr')
                service_result['results'] = []
                for row in table_results:
                    try: href = row.find('a')['href']
                    except TypeError:
                        href = ''
                    if re.search('criterion.*', str(href), re.IGNORECASE):
                        for cell in row:
                            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                            except:
                                cell_text = ''
                            point = ''
                            result = ''
                            if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                                 point = cell_text
                            if re.search('.*met$', cell_text, re.IGNORECASE):
                                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                                    result = 'did not meet'
                                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                                    result = 'met'
                            if point != '' and result != '' and re.search('.*met|meet$', result, re.IGNORECASE): 
                                service_result['results'].append({'point':point,'result':result})
                service['results'] = service_result['results']
                del service_result
                del content



In [ ]:
for service in first_service_results:
    if service["results"] == []:    
            service_result = {"service_details":[],'results':[],'html_contet':[]}
            for service_detail in service_details:
                print(service_detail)

In [ ]:
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
# #Get results from even older tables
# # where the links in the table are NOT helpfully numbered
for service in first_service_results:
    if service["results"] == []:
        service_result = {"service_details":[],'results':[]}
        for service_detail in service_details:
            if service_detail['service'] == service['service_details']['service']:  
                print(service['service_details']['service'])          
                content = service['html_content']
                get_service_attributes(content,service_detail)
                service_result['service_details'] = service_detail
                table_results = content.find_all('tr')
                service_result['results'] = []
                for row in table_results:
                    try: href = row.find('a')['href']
                    except TypeError:
                        href = ''
                    if re.search('criterion.*', str(href), re.IGNORECASE):
                        for cell in row:
                            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                            except:
                                cell_text = ''
                            point = ''
                            result = ''
                            if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                                point = cell_text
                            if re.search('.*met$', cell_text, re.IGNORECASE):
                                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                                    result = 'did not meet'
                                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                                    result = 'met'
                                    print('point '+point+'result '+result)
                            if point != '' or result != '' or re.search('.*met|meet$', result, re.IGNORECASE): 
                                service_result['results'].append({'point':point,'result':result})
                service['results'] = service_result['results']
                del service_result
                del content

In [ ]:
for service in first_service_results:
    print(service)
    print(service['results'])

In [ ]:
    point=''
    result = ''
    collect_next_cell = ''
    service_result = {"service_details":[],'results':[]}
    extension = service_detail['attributes']['url']
    URL = 'https://www.gov.uk' +  '/service-standard-reports/electronic-visa-waiver-alpha-assessment'
    page = requests.get(URL)
    content = BeautifulSoup(page.content, 'html.parser')
    results = content.find_all('p')
    paragraphs = content.find_all('p')
    for paragraph in paragraphs:
     text = paragraph.text.replace('\n', '').replace('\r', '').strip()
     if re.search('^Date of Assessment:.*', text, re.IGNORECASE):
            print('assessment_date')
            assessment_date = re.search('^(Date of Assessment:)(.*)', text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_date'] = assessment_date
     if re.search('^Assessment stage:.*', text, re.IGNORECASE):
            print('assessment_stage')
            assessment_stage = re.search('^(Assessment stage:)(.*)', text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_stage'] = assessment_stage
     if re.search('^Result of Assessment:.*', text, re.IGNORECASE):
            print('assessment_result')
            assessment_result = re.search('^(Result of Assessment:)(.*)', text, re.IGNORECASE).group(2)
            service_detail['attributes']['assessment_result'] = assessment_result
     if re.search('^Department / Agency:.*', text, re.IGNORECASE):
            print('service_provider')            
            service_provider = ''
            service_provider = re.search('^(Department / Agency:)(.*)', text, re.IGNORECASE).group(2)
            service_detail['attributes']['service_provider'] = service_provider
    service_result['service_details'] = service_detail
    table_results = content.find_all('tr')
    for row in table_results:
        row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
        for cell in row:
            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
            except:
                cell_text = ''
            if re.search('^\d{1,2}$', cell_text, re.IGNORECASE):
                point = cell_text
                collect_next_cell = True
            if collect_next_cell and not re.search('^\d{1,2}$', cell_text, re.IGNORECASE) and cell_text != '' and re.search('.*Met|.*Yes|.*No|.*Pass|.*Fail$', cell_text, re.IGNORECASE): 
                collect_next_cell = False
                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                    result = 'did not meet'           
                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                        result = 'met'
                else:
                    result = cell_text 
                service_result['results'].append({'point':point,'result':result})


In [ ]:

      second_service_results     


In [ ]:
import pandas as pd
service_result_df = pd.DataFrame()
for service_result in service_results:
    service_name = service_result['service_details']['service']
    service_stage = service_result['service_details']['attributes']['assessment_stage']
    assessment_date = service_result['service_details']['attributes']['assessment_date']
    service_provider = service_result['service_details']['attributes']['service_provider']
    results = service_result['results']
    if len(results)>0:
        service_result_df_raw = pd.DataFrame.from_dict(results).drop_duplicates(subset=["point"], keep='last').set_index('point')
        service_result_df_wide = service_result_df_raw.T
        service_result_df_wide['service_name'] = service_name
        service_result_df_wide['service_stage'] = service_stage
        service_result_df_wide['service_provider'] = service_provider
        service_result_df_wide['assessment_date'] = assessment_date
        service_result_df_wide.set_index('service_name')
        service_result_df = service_result_df.append(service_result_df_wide)
        service_result_df.sort_values(by=['assessment_date','service_name','assessment_date'])
service_result_df.drop_duplicates()
cols = service_result_df.columns.tolist()

In [ ]:
cols = [int(ele) if ele.isdigit() else ele for ele in cols] 
# comparator function for sort 
def mixs(num): 
    try: 
        ele = int(num) 
        return (0, ele, '') 
    except ValueError: 
        return (1, num, '') 

cols.sort(key = mixs) 
cols = cols[-4:] + cols[:-4]
cols = [str(i) for i in cols]
service_result_df = service_result_df[cols]
service_result_df = service_result_df.set_index('service_name')